In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
info = pd.read_csv('data/snp_info.csv')

In [3]:
train.drop(['id', 'father', 'mother', 'gender'], axis = 1, inplace = True)

In [4]:
test.drop(['id', 'father', 'mother', 'gender'], axis = 1, inplace = True)

In [10]:
snp = train.columns[1:16]
snp_data = []

for col in snp :
    snp_data += list(train[col].values)

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
class_le = LabelEncoder()
snp_le = LabelEncoder()

In [13]:
train['class'] = class_le.fit_transform(train['class'])

In [16]:
snp_le.fit(snp_data)

LabelEncoder()

In [20]:
for col in train.columns[:16]:
    if col in snp:
        train[col] = snp_le.transform(train[col])
        test[col] = snp_le.transform(test[col])

In [23]:
reg = setup(train, target = 'class')

,Description,Value
0,session_id,8114
1,Target,class
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(262, 17)"
5,Missing Values,False
6,Numeric Features,15
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [24]:
top3 = compare_models(n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9447,0.9871,0.9488,0.9483,0.9443,0.9151,0.9175,0.0280
et,Extra Trees Classifier,0.9342,0.9847,0.9363,0.9397,0.9332,0.8985,0.9025,0.0250
ridge,Ridge Classifier,0.9281,0.0000,0.9321,0.9335,0.9273,0.8889,0.8927,0.0020
lda,Linear Discriminant Analysis,0.9281,0.9873,0.9321,0.9335,0.9273,0.8889,0.8927,0.0030
lr,Logistic Regression,0.9234,0.9883,0.9296,0.9306,0.9226,0.8829,0.8876,0.6080
lightgbm,Light Gradient Boosting Machine,0.9178,0.9759,0.9240,0.9222,0.9167,0.8735,0.8769,0.0070
gbc,Gradient Boosting Classifier,0.9123,0.9809,0.9171,0.9188,0.9104,0.8649,0.8702,0.0230
knn,K Neighbors Classifier,0.9006,0.9552,0.9028,0.9143,0.8945,0.8436,0.8562,0.3190
svm,SVM - Linear Kernel,0.8845,0.0000,0.8917,0.9004,0.8811,0.8232,0.8344,0.0040
nb,Naive Bayes,0.8842,0.9808,0.8849,0.9116,0.8739,0.8193,0.8398,0.2480


In [25]:
tuned_top3 = [tune_model(i) for i in top3]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9474,0.0000,0.9444,0.9532,0.9465,0.9188,0.9228
2,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.8889,0.0000,0.8889,0.9111,0.8840,0.8235,0.8407
4,0.7778,0.0000,0.8056,0.7778,0.7778,0.6538,0.6538
5,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9444,0.0000,0.9444,0.9506,0.9436,0.9126,0.9172
7,0.9444,0.0000,0.9444,0.9514,0.9438,0.9155,0.9199
8,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [26]:
blend_top3 = blend_models(estimator_list = tuned_top3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9474,0.0000,0.9444,0.9532,0.9465,0.9188,0.9228
2,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9444,0.0000,0.9444,0.9506,0.9436,0.9126,0.9172
4,0.8333,0.0000,0.8472,0.8346,0.8307,0.7379,0.7415
5,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9444,0.0000,0.9444,0.9506,0.9436,0.9126,0.9172
7,0.9444,0.0000,0.9444,0.9514,0.9438,0.9155,0.9199
8,0.9444,0.0000,0.9444,0.9514,0.9438,0.9155,0.9199


In [27]:
model = finalize_model(blend_top3)

In [28]:
pred = predict_model(model, data = test)

In [32]:
pred['Label']

0      0
1      1
2      2
3      1
4      0
      ..
170    1
171    2
172    2
173    1
174    1
Name: Label, Length: 175, dtype: int32

In [29]:
submit = pd.read_csv('data/sample_submission.csv')

In [33]:
submit['class'] = class_le.inverse_transform(pred['Label'])

In [34]:
submit

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B


In [35]:
submit.to_csv('data/submit/submit_pycaret.csv', index=False)